In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.models import Model, load_model
from keras.layers import LSTM, Dense, TimeDistributed, Input, Masking, RepeatVector, Bidirectional, Embedding
nltk.download('punkt')
import string
punc = list(string.punctuation)
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))
print(stop_words)

Using TensorFlow backend.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.metrics import pairwise
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pickle
import csv
import os
from scipy.spatial.distance import cosine
import time
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.feature_extraction.text import CountVectorizer
import torch

In [3]:
## HYPER-PARAMETERS

num_words = 10000
MAX_SEQUENCE_LENGTH = 16
WORD_EMBEDDING_DIM = 200
batch_size = 640
ENCODING_DIM = 500
NUM_DATAPOINT = -1 # All in the file
NUM_ICD_CODES = 2833
NUM_EPOCHS = 2
VALIDATION_SPLIT = 0.2
PATIENCE = 5
TOP_K_PER_SENTENCE = 10
similarity_threshold = 0.7

THRESHOLD = -1 # TODO

ICD_DESC_FILENAME = 'ICD_desc_with_freq.csv'
#DD_FILENAME = 'MIMIC_III_Final_1000_clean_shorted.csv'
#DD_FILENAME = 'MIMIC-III-Final_5000_clean_shortened.csv'
DD_FILENAME = 'MIMIC-III-Final_cleaned.csv'
EMB_FILE = 'PMC-w2v.bin'

#RESULT_FILENAME = 'Experiment Results.xlsx'
#http://yaronvazana.com/2019/09/28/training-an-autoencoder-to-generate-text-embeddings/

In [4]:
#Calculate Sentence Embedding using Average Word Embedding of tokens
def get_sentence_embedding(sent, embedding):
    words = word_tokenize(sent)
    m = len(words)
    total_embedding = 0
    
    for i in range(m):
        try:
            w = embedding[words[i]]
        except:
            w = embedding['unk']
        
        total_embedding += w
    
    try:
        return total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True)
    except:
        #print('sent : ', sent)
        #print('m: ', m)
        return np.array(total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True))

In [5]:
#Calculate Word Embedding
def get_word_embedding(word, embedding):
    try:
        return embedding[word]
    except:
        return embedding['unk']

In [6]:
#Load Word Embedding (Pub Med W2V)
from gensim.models.keyedvectors import KeyedVectors
word_model = KeyedVectors.load_word2vec_format(EMB_FILE, binary=True)
print('Found %s word vectors.' % len(word_model.index2word))
print(word_model['Cholera'].shape)

Found 2515686 word vectors.
(200,)


In [7]:
word_model['Cholera']/np.linalg.norm(word_model['Cholera'], 2)

array([-0.00481831,  0.13864845, -0.0831728 , -0.00180067, -0.00696073,
       -0.02875327,  0.06748949,  0.081326  ,  0.11438117,  0.00530638,
       -0.05866855,  0.05307228, -0.09064269,  0.03729252, -0.00409504,
        0.02162931,  0.11918882, -0.05017674,  0.03869108,  0.12593974,
       -0.06599744,  0.13589439,  0.02106961, -0.02205387,  0.04005024,
        0.11348598, -0.09267536, -0.04086379, -0.07475508,  0.02114647,
        0.03469999,  0.02900406,  0.06926779,  0.11697073, -0.01642677,
       -0.00433   ,  0.06071504, -0.1460445 , -0.01645673, -0.0205683 ,
        0.01536769,  0.0160432 , -0.00429431,  0.03412509, -0.05015106,
        0.0433115 ,  0.08293187,  0.14879459, -0.0841924 , -0.06157425,
       -0.02435092, -0.05808348, -0.11018413,  0.01283847, -0.07155055,
        0.08904947, -0.0320092 , -0.11904893, -0.01103841,  0.11339661,
        0.01087772, -0.05354536,  0.0755807 ,  0.07982337,  0.11575826,
       -0.1831625 ,  0.01570357, -0.05355467,  0.065675  , -0.05

# ICD File

In [8]:
ICD_desc_filename = ICD_DESC_FILENAME
df_icd = pd.read_csv(ICD_desc_filename, encoding = 'latin1')

df_icd = df_icd[['Code', 'Long Description']]

print(df_icd.shape)

print('\n', df_icd.head())

(6553, 2)

     Code                                   Long Description
0   4019                 Unspecified essential hypertension
1   4280              Congestive heart failure, unspecified
2  42731                                Atrial fibrillation
3  41401  Coronary atherosclerosis of native coronary ar...
4   5849                  Acute kidney failure, unspecified


In [9]:
#Clean the ICD Description Texts

df_icd_cleaned = df_icd

#Delete df_icd
del df_icd

def clean_icd(x):
    sent = x
    sent_tokenized = word_tokenize(sent)
    
    clean_sent = ''
    
    for token in sent_tokenized:
        if token.lower() not in punc and token.lower() not in stop_words:
            clean_sent = clean_sent + ' ' + token.lower()
    
    x = clean_sent
    
    del clean_sent
    
    return x


df_icd_cleaned['Long Description'] = df_icd_cleaned.apply(lambda x: clean_icd(x['Long Description']), axis = 1)

df_icd_cleaned.head()

,Code,Long Description
0,4019,unspecified essential hypertension
1,4280,congestive heart failure unspecified
2,42731,atrial fibrillation
3,41401,coronary atherosclerosis native coronary artery
4,5849,acute kidney failure unspecified


In [10]:
#Capture the ICD embedding as a numpy matrix
#icd_sent_emb = df_icd_cleaned['Sentence Embedding'][:]
#icd_sent_emb_list = [x for x in icd_sent_emb]

#icd_sent_emb_matrix = np.asarray(icd_sent_emb_list, dtype=np.float32)
#icd_sent_emb_matrix.shape

In [11]:
icd_sentences = list(df_icd_cleaned.head(NUM_ICD_CODES)['Long Description']) # Number of sentences
icd_codes = list(df_icd_cleaned.head(NUM_ICD_CODES)['Code'])
icd_index = list(range(NUM_ICD_CODES))

print(len(icd_sentences), len(icd_codes), len(icd_index))
print("ICD_Sentences = ", len(icd_sentences))

2833 2833 2833
ICD_Sentences =  2833


In [12]:
# 3 Dictionary of ICD index, ICD Code and ICD desc

# index, codes
dict_index_code = dict(zip(icd_index, icd_codes))

# codes, index
dict_code_index = dict(zip(icd_codes, icd_index))

#index, code desc
dict_index_desc = dict(zip(icd_index, icd_sentences))

#icd sentence embedding matrix slice
#icd_sent_emb_matrix = icd_sent_emb_matrix[:NUM_ICD_CODES]

# Cleaned MIMIC-III DD file

In [13]:
#Load the DD File and Set the range of records to process

DD_filename = DD_FILENAME
df_DD = pd.read_csv(DD_filename)
df_DD = df_DD[30000:]
df_DD.reset_index(drop=True, inplace= True)
FILESIZE = df_DD.shape[0]
df_DD.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,clean_text,top_ICD
0,30000,{Admission Date: [**2134-4-21**] ...,156925.0,"['4412', '99859', '5601', '7464', '5119', '518...",admission date discharge date of birth sex nam...,"['4412', '99859', '5601', '7464', '5119', '518..."
1,30001,{Admission Date: [**2169-1-26**] Discha...,156930.0,"['80162', '80605', 'E8810', '185', '2720']",admission date discharge of birth sex trauma s...,"['80605', 'E8810', '185', '2720']"
2,30002,{Admission Date: [**2139-10-13**] ...,156931.0,"['9961', '99812', '2866', '4280', '5185', '584...",admission date discharge date of birth sex nam...,"['9961', '99812', '2866', '4280', '5845', '427..."
3,30003,{Admission Date: [**2116-7-21**] ...,156932.0,"['55090', '7895', '2851', '51882', '4280', '58...",admission date discharge date of birth sex doc...,"['55090', '2851', '51882', '4280', '5845', '28..."
4,30004,{Admission Date: [**2186-10-11**] ...,156933.0,"['53784', '2800', '4019']",admission date discharge date recorded as havi...,"['53784', '2800', '4019']"


In [14]:
print(FILESIZE)

22722


In [15]:
DD_Data = df_DD[['ICD9_CODE', 'clean_text']]
del df_DD
DD_Data

,ICD9_CODE,clean_text
0,"['4412', '99859', '5601', '7464', '5119', '518...",admission date discharge date of birth sex nam...
1,"['80162', '80605', 'E8810', '185', '2720']",admission date discharge of birth sex trauma s...
2,"['9961', '99812', '2866', '4280', '5185', '584...",admission date discharge date of birth sex nam...
3,"['55090', '7895', '2851', '51882', '4280', '58...",admission date discharge date of birth sex doc...
4,"['53784', '2800', '4019']",admission date discharge date recorded as havi...
...,...,...
22717,"['41031', '42821', '42731', '4271', '5180', '4...",admission date discharge date of birth sex com...
22718,"['486', '4280', '51881', '3970', '496', '4169'...",admission date discharge date of birth sex of ...
22719,"['4210', '7464', '42971', '30401', '4412', '44...",admission date discharge date of birth sex las...
22720,"['41401', '9971', '9975', '42731', '78820', '4...",admission date discharge date of birth sex nam...


In [16]:
#Calculate the Corpus Statistics of the DD Data

Vocab = {}

doc_len = []

for index in range(DD_Data.shape[0]):
    sent = word_tokenize(DD_Data['clean_text'][index])
    
    doc_len.append(len(sent))
    
    for word in sent:
        if word in Vocab.keys():
            Vocab[word]+=1
        else:
            Vocab[word]=1

print('Vocab Size: ', len(Vocab.keys()))
print('Avg Doc Len: ', float(sum(doc_len))/float(len(doc_len)))
del Vocab, doc_len

Vocab Size:  44076
Avg Doc Len:  1305.4134319162047


In [17]:
#Get TF vector using entire corpus
def get_tf(docs):
    
    vectorizer = CountVectorizer()#CountVectorizer(stop_words = 'english')
    X = vectorizer.fit_transform(docs)#(docs['sentence'])

    # place tf values in a pandas data frame
    tf_vecs = pd.DataFrame(X.T.todense(), index = vectorizer.get_feature_names())
    
    #Take sum accross sentences
    freq_vec = tf_vecs.sum(axis = 1)
    freq_vec = pd.DataFrame(freq_vec, index=freq_vec.index, columns=['Frequency'])
    
    return vectorizer, freq_vec

In [18]:
#remove terms not present in freq_vec
def remove_unwanted_words(words, freq_vec):
    
    pruned_words = [x for x in words if x in freq_vec.index]
    
    #for i in words:
    #    if i in freq_vec.index:
    #        pruned_words.append(i)
    
    return list(set(pruned_words))

In [19]:
#Function to Generate Importance Vector Vector
def get_importance_vector(sent, freq_vec):

    #tokenize sentence
    words = list(set(word_tokenize(sent)))
    
    #remove terms not occuring in freq_vec
    pruned_words = remove_unwanted_words(words, freq_vec)
    
    #get the slice of freq_vec using the remaining terms
    #print(words, pruned_words)
    freq_slice = freq_vec.loc[pruned_words, :]
    
    #L1 normalize to obtain importance vector
    imp_vec = freq_slice/np.linalg.norm(freq_slice, ord = 1)
    
    return imp_vec, pruned_words

In [20]:
#calculate the cosine similarity matrix for 2 sentences
def cosine_sim_matrix(s1, s2):
    
    #Get token-wise embedding
    s1_token_emb = np.array([get_word_embedding(word, word_model) for word in s1])
    #Normalize
    s1_token_emb/=np.linalg.norm(s1_token_emb, axis = 1, ord = 2, keepdims=True)
    
    #Get token-wise embedding of icd text
    s2_token_emb = np.array([get_word_embedding(word, word_model) for word in s2])
    #Normalize
    s2_token_emb/=np.linalg.norm(s2_token_emb, axis = 1, ord = 2, keepdims=True)
    
    
    sim_matrix = s1_token_emb.dot(s2_token_emb.T)
    
    return np.clip(sim_matrix, -1.0, 1.0) #Clip the Elements of the matrix

In [21]:
#Reset Debugging Counter
k = 0
#Calculate the Similarity Score between 2 senteces
def RAMN_similarity(sent_1, sent_2, model, freq_vec):
    global k
    try:
        
        #get importance vector and list of words
        #start = time.time()
        sent1_imp_vec, sent1 = get_importance_vector(sent_1, freq_vec)
        sent2_imp_vec, sent2 = get_importance_vector(sent_2, freq_vec)
        
        #end = time.time()
        #print('Imp Vector Time: ', end - start)
        
        #Compute Cosine Similarity Matrix
        #start = time.time()
        
        sim_mat = cosine_sim_matrix(sent1, sent2)
        #end = time.time()
        #print('Sim Matrix Time: ', end - start)
        
        #start = time.time()

        #q->Q
        max_vec_q = sim_mat.max(axis = 1)
        dq = sent1_imp_vec
        for i in range(len(sent1)):
            wi = sent1[i]

            #Mask the matching terms
            if wi in sent2:
                dq.loc[wi,'Frequency'] = 1.0
            else:
                pass

        #Multiply dq with max similarity vector
        g_w_Q = dq.to_numpy().transpose() * max_vec_q

        #Take elementwise product to obtain final similarity - q -> Q
        g_q_Q = np.prod(g_w_Q)

        #Q->q
        max_vec_Q = sim_mat.max(axis = 0).transpose()
        dQ = sent2_imp_vec
        for j in range(len(sent2)):
            wj = sent2[j]

            #Mask the matching terms
            if wj in sent1:
                dQ.loc[wj,'Frequency'] = 1.0
            else:
                pass

        #Multiply dQ with max similarity vector
        g_w_q = dQ.to_numpy().transpose() * max_vec_Q

        #Take elementwise product to obtain final similarity - Q -> q
        g_Q_q = np.prod(g_w_q)
        
        #end = time.time()
        #print('Sim Time: ', end - start)
        
        #k = k + 1
        #print('k: ', k)
        
        del sim_mat, sent1_imp_vec, sent2_imp_vec, sent1, sent2
        
        
        #Sample Output
        #print('sent1_imp_vec: ', sent1_imp_vec)
        #print('sent2_imp_vec: ', sent2_imp_vec)
        #print('sim_mat : ', sim_mat)
        

        return max(g_q_Q, g_Q_q)#g_q_Q * g_Q_q
    except:
        #print('sent_1: ', sent_1)
        sent_1_emb = get_sentence_embedding(sent_1, word_model)
        #print('sent_2: ', sent_2)
        sent_2_emb = get_sentence_embedding(sent_2, word_model)
        
        #k = k + 1
        #print('k: ', k)
        
        return sent_1_emb.dot(sent_2_emb.T)#cosine_sim(sent_1_emb, sent_2_emb)

In [22]:
#Reset Debugging Counter
k = 0
#Calculate the Similarity Score between 2 senteces
def RAMN_similarity(sent_1, sent_2, model, freq_vec):
    global k
    try:
        
        #get importance vector and list of words
        #start = time.time()
        sent1_imp_vec, sent1 = get_importance_vector(sent_1, freq_vec)
        sent2_imp_vec, sent2 = get_importance_vector(sent_2, freq_vec)
        
        #end = time.time()
        #print('Imp Vector Time: ', end - start)
        
        #Compute Cosine Similarity Matrix
        #start = time.time()
        
        sim_mat = cosine_sim_matrix(sent1, sent2)
        #end = time.time()
        #print('Sim Matrix Time: ', end - start)
        
        #start = time.time()

        #q->Q
        max_vec_q = sim_mat.max(axis = 1)
        dq = sent1_imp_vec
        
        #Q->q
        max_vec_Q = sim_mat.max(axis = 0).transpose()
        dQ = sent2_imp_vec
        
        #Mask the matching terms
        for i in range(len(sent1)):
            wi = sent1[i]
            
            if wi in sent2:
                dq.loc[wi,'Frequency'] = 1.0
                dQ.loc[wi,'Frequency'] = 1.0
            else:
                pass

        #Multiply dq with max similarity vector
        g_w_Q = dq.to_numpy().transpose() * max_vec_q

        #Take elementwise product to obtain final similarity - q -> Q
        g_q_Q = np.prod(g_w_Q)
        

        #Multiply dQ with max similarity vector
        g_w_q = dQ.to_numpy().transpose() * max_vec_Q

        #Take elementwise product to obtain final similarity - Q -> q
        g_Q_q = np.prod(g_w_q)
        
        #end = time.time()
        #print('Sim Time: ', end - start)
        
        #k = k + 1
        #print('k: ', k)
        
        del sim_mat, sent1_imp_vec, sent2_imp_vec, sent1, sent2
        
        
        #Sample Output
        #print('sent1_imp_vec: ', sent1_imp_vec)
        #print('sent2_imp_vec: ', sent2_imp_vec)
        #print('sim_mat : ', sim_mat)
        

        return max(g_q_Q, g_Q_q)#g_q_Q * g_Q_q
    except:
        #print('sent_1: ', sent_1)
        sent_1_emb = get_sentence_embedding(sent_1, word_model)
        #print('sent_2: ', sent_2)
        sent_2_emb = get_sentence_embedding(sent_2, word_model)
        
        #k = k + 1
        #print('k: ', k)
        
        return sent_1_emb.dot(sent_2_emb.T)#cosine_sim(sent_1_emb, sent_2_emb)

In [23]:
#Convert ICD Indices to hot vectors of length: NUM_ICD_CODE
def generate_binary(icd_index_list, total_icd = NUM_ICD_CODES):
    binary_icd = np.zeros(total_icd)
    for x in icd_index_list:
        binary_icd[x] = 1
        
    return binary_icd

In [24]:
#Calculate Specificity

def calc_specificity(pred_vec, act_vec):
    m = len(pred_vec)
    
    specificity = []
    
    for i in range(m):
        pred, act = pred_vec[i], act_vec[i]
        dict_XY = {'pred':pred, 'act':act}
        res = pd.DataFrame(dict_XY, columns=['pred', 'act'])
        res['Sum'] = res['pred'] + res['act']
        
        pred_p = res[res.pred == 1]
        pred_n = res[res.act == 1]
        
        tp = len(res[res.Sum == 2])
        tn = len(res[res.Sum == 0]) 
        
        fp = len(pred_p[pred_p.act == 0])
        fn = len(pred_n[pred_n.act == 1])
        
        #Calculate specificity for this example:
        spec = tn/(tn + fp)
        
        specificity.append(spec)
    
    return sum(specificity) / float(m)
        

In [25]:
#Predict ICD 9 Codes


#construct the TF Vevtor using the entire corpus
c_vectorizer, freq_vec = get_tf(DD_Data['clean_text'])

#Calculate the importance vector for all icd texts:
icd_feature_vecs = {icd_text : get_importance_vector(icd_text, freq_vec) for icd_text in icd_sentences}

start_time = time.time()

#Calculate the Similarity of each document with each icd text
X = DD_Data['clean_text']
Y = list(DD_Data['ICD9_CODE'])
m = len(Y)

#initialize similarity cache
sim = []

for i in range(m):
    text = X[i]
    #Modified#Calculate cosine similarity of sentence with icd sentences
    sim.append(np.array([RAMN_similarity(icd_text, text, word_model, freq_vec) for icd_text in icd_sentences]))#cosine_sim1_matrix(sent_emb)
    
    #reset debugging counter
    #global k
    #k = 0
    
    #Create Similarity File with first 100 records
    if (i+1) == 100:
        sim_df = pd.DataFrame(sim, dtype = float).to_csv('Similarity_Data/Optimized_NP_RAMN_similarity_data_' + str(m) + '_records.csv')
        sim = []
        print('Similarity File Created..')
        print('\nTime elapsed: ', (time.time() - start_time)/60.0, ' minutes')

    #reset debugging counter
    #global k
    #k = 0
    
    if (i+1)%10 == 0 or (i+1) == m:
        print(str(i+1) + '/' + str(m) + ' records done.')
    
    if (i+1)%100 == 0 and i+1 != 100 or i+1 == m:
        #Append the next 100 records to the existing file
        sim_df = pd.DataFrame(sim, dtype = float).to_csv('Similarity_Data/Optimized_NP_RAMN_similarity_data_' + str(m) + '_records.csv', mode = 'a', header=False)
        #reset similarity cache
        sim = []
        print('Similarity File Refreshed..')
        print('\nTime elapsed: ', (time.time() - start_time)/60.0, ' minutes')    
        
time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')

10/22722 records done.
20/22722 records done.
30/22722 records done.
40/22722 records done.
50/22722 records done.
60/22722 records done.
70/22722 records done.
80/22722 records done.
90/22722 records done.
Similarity File Created..

Time elapsed:  77.67649606863658  minutes
100/22722 records done.
110/22722 records done.
120/22722 records done.
130/22722 records done.
140/22722 records done.
150/22722 records done.
160/22722 records done.
170/22722 records done.
180/22722 records done.
190/22722 records done.
200/22722 records done.
Similarity File Refreshed..

Time elapsed:  157.97813770373662  minutes
210/22722 records done.
220/22722 records done.
230/22722 records done.
240/22722 records done.
250/22722 records done.
260/22722 records done.
270/22722 records done.
280/22722 records done.
290/22722 records done.
300/22722 records done.
Similarity File Refreshed..

Time elapsed:  234.5182651201884  minutes
310/22722 records done.
320/22722 records done.
330/22722 records done.
340/2

2610/22722 records done.
2620/22722 records done.
2630/22722 records done.
2640/22722 records done.
2650/22722 records done.
2660/22722 records done.
2670/22722 records done.
2680/22722 records done.
2690/22722 records done.
2700/22722 records done.
Similarity File Refreshed..

Time elapsed:  2188.4202180862426  minutes
2710/22722 records done.
2720/22722 records done.
2730/22722 records done.
2740/22722 records done.
2750/22722 records done.
2760/22722 records done.
2770/22722 records done.
2780/22722 records done.
2790/22722 records done.
2800/22722 records done.
Similarity File Refreshed..

Time elapsed:  2268.3219585220018  minutes
2810/22722 records done.
2820/22722 records done.
2830/22722 records done.
2840/22722 records done.
2850/22722 records done.
2860/22722 records done.
2870/22722 records done.
2880/22722 records done.
2890/22722 records done.
2900/22722 records done.
Similarity File Refreshed..

Time elapsed:  2353.7260490059853  minutes
2910/22722 records done.
2920/2272

5180/22722 records done.
5190/22722 records done.
5200/22722 records done.
Similarity File Refreshed..

Time elapsed:  4245.71208867232  minutes
5210/22722 records done.
5220/22722 records done.
5230/22722 records done.
5240/22722 records done.
5250/22722 records done.
5260/22722 records done.
5270/22722 records done.
5280/22722 records done.
5290/22722 records done.
5300/22722 records done.
Similarity File Refreshed..

Time elapsed:  4344.235101234913  minutes
5310/22722 records done.
5320/22722 records done.
5330/22722 records done.
5340/22722 records done.
5350/22722 records done.
5360/22722 records done.
5370/22722 records done.
5380/22722 records done.
5390/22722 records done.
5400/22722 records done.
Similarity File Refreshed..

Time elapsed:  4425.304619268576  minutes
5410/22722 records done.
5420/22722 records done.
5430/22722 records done.
5440/22722 records done.
5450/22722 records done.
5460/22722 records done.
5470/22722 records done.
5480/22722 records done.
5490/22722 re

7730/22722 records done.
7740/22722 records done.
7750/22722 records done.
7760/22722 records done.
7770/22722 records done.
7780/22722 records done.
7790/22722 records done.
7800/22722 records done.
Similarity File Refreshed..

Time elapsed:  6405.65927460591  minutes
7810/22722 records done.
7820/22722 records done.
7830/22722 records done.
7840/22722 records done.
7850/22722 records done.
7860/22722 records done.
7870/22722 records done.
7880/22722 records done.
7890/22722 records done.
7900/22722 records done.
Similarity File Refreshed..

Time elapsed:  6480.679940668741  minutes
7910/22722 records done.
7920/22722 records done.
7930/22722 records done.
7940/22722 records done.
7950/22722 records done.
7960/22722 records done.
7970/22722 records done.
7980/22722 records done.
7990/22722 records done.
8000/22722 records done.
Similarity File Refreshed..

Time elapsed:  6559.110912934939  minutes
8010/22722 records done.
8020/22722 records done.
8030/22722 records done.
8040/22722 re

10290/22722 records done.
10300/22722 records done.
Similarity File Refreshed..

Time elapsed:  8453.019699235758  minutes
10310/22722 records done.
10320/22722 records done.
10330/22722 records done.
10340/22722 records done.
10350/22722 records done.
10360/22722 records done.
10370/22722 records done.
10380/22722 records done.
10390/22722 records done.
10400/22722 records done.
Similarity File Refreshed..

Time elapsed:  8530.523574034372  minutes
10410/22722 records done.
10420/22722 records done.
10430/22722 records done.
10440/22722 records done.
10450/22722 records done.
10460/22722 records done.
10470/22722 records done.
10480/22722 records done.
10490/22722 records done.
10500/22722 records done.
Similarity File Refreshed..

Time elapsed:  8618.661412151654  minutes
10510/22722 records done.
10520/22722 records done.
10530/22722 records done.
10540/22722 records done.
10550/22722 records done.
10560/22722 records done.
10570/22722 records done.
10580/22722 records done.
10590/2

12760/22722 records done.
12770/22722 records done.
12780/22722 records done.
12790/22722 records done.
12800/22722 records done.
Similarity File Refreshed..

Time elapsed:  10493.999862722556  minutes
12810/22722 records done.
12820/22722 records done.
12830/22722 records done.
12840/22722 records done.
12850/22722 records done.
12860/22722 records done.
12870/22722 records done.
12880/22722 records done.
12890/22722 records done.
12900/22722 records done.
Similarity File Refreshed..

Time elapsed:  10570.446099853516  minutes
12910/22722 records done.
12920/22722 records done.
12930/22722 records done.
12940/22722 records done.
12950/22722 records done.
12960/22722 records done.
12970/22722 records done.
12980/22722 records done.
12990/22722 records done.
13000/22722 records done.
Similarity File Refreshed..

Time elapsed:  10655.20403683583  minutes
13010/22722 records done.
13020/22722 records done.
13030/22722 records done.
13040/22722 records done.
13050/22722 records done.
13060

15230/22722 records done.
15240/22722 records done.
15250/22722 records done.
15260/22722 records done.
15270/22722 records done.
15280/22722 records done.
15290/22722 records done.
15300/22722 records done.
Similarity File Refreshed..

Time elapsed:  12470.653870050113  minutes
15310/22722 records done.
15320/22722 records done.
15330/22722 records done.
15340/22722 records done.
15350/22722 records done.
15360/22722 records done.
15370/22722 records done.
15380/22722 records done.
15390/22722 records done.
15400/22722 records done.
Similarity File Refreshed..

Time elapsed:  12542.385421419143  minutes
15410/22722 records done.
15420/22722 records done.
15430/22722 records done.
15440/22722 records done.
15450/22722 records done.
15460/22722 records done.
15470/22722 records done.
15480/22722 records done.
15490/22722 records done.
15500/22722 records done.
Similarity File Refreshed..

Time elapsed:  12618.816241939863  minutes
15510/22722 records done.
15520/22722 records done.
1553

17710/22722 records done.
17720/22722 records done.
17730/22722 records done.
17740/22722 records done.
17750/22722 records done.
17760/22722 records done.
17770/22722 records done.
17780/22722 records done.
17790/22722 records done.
17800/22722 records done.
Similarity File Refreshed..

Time elapsed:  14408.314183469613  minutes
17810/22722 records done.
17820/22722 records done.
17830/22722 records done.
17840/22722 records done.
17850/22722 records done.
17860/22722 records done.
17870/22722 records done.
17880/22722 records done.
17890/22722 records done.
17900/22722 records done.
Similarity File Refreshed..

Time elapsed:  14498.023637453714  minutes
17910/22722 records done.
17920/22722 records done.
17930/22722 records done.
17940/22722 records done.
17950/22722 records done.
17960/22722 records done.
17970/22722 records done.
17980/22722 records done.
17990/22722 records done.
18000/22722 records done.
Similarity File Refreshed..

Time elapsed:  14575.922210252285  minutes
1801

20200/22722 records done.
Similarity File Refreshed..

Time elapsed:  16245.716834501425  minutes
20210/22722 records done.
20220/22722 records done.
20230/22722 records done.
20240/22722 records done.
20250/22722 records done.
20260/22722 records done.
20270/22722 records done.
20280/22722 records done.
20290/22722 records done.
20300/22722 records done.
Similarity File Refreshed..

Time elapsed:  16318.945293970903  minutes
20310/22722 records done.
20320/22722 records done.
20330/22722 records done.
20340/22722 records done.
20350/22722 records done.
20360/22722 records done.
20370/22722 records done.
20380/22722 records done.
20390/22722 records done.
20400/22722 records done.
Similarity File Refreshed..

Time elapsed:  16397.881398634116  minutes
20410/22722 records done.
20420/22722 records done.
20430/22722 records done.
20440/22722 records done.
20450/22722 records done.
20460/22722 records done.
20470/22722 records done.
20480/22722 records done.
20490/22722 records done.
2050

22670/22722 records done.
22680/22722 records done.
22690/22722 records done.
22700/22722 records done.
Similarity File Refreshed..

Time elapsed:  18215.000308636823  minutes
22710/22722 records done.
22720/22722 records done.

Time elapsed:  18232.260480070116  minutes


In [35]:
#Load Similarity Data
sim = pd.read_csv('Similarity_Data/Optimized_NP_RAMN_similarity_data_' + str(m) + '_records.csv')
#sim = pd.read_csv('Optimized_NP_RAMN_similarity_data_' + str(m) + '_records.csv')


#Drop first column
sim = sim.drop(['Unnamed: 0'], axis = 1)

#Convert DataFrame to List
sim = np.array(sim.values.tolist())

In [36]:
len(sim)

22722

In [37]:
#Save Predictions and Corresponding Scores per threshold

#Initialise Similarity Result Dataframe
results_sim = pd.DataFrame(columns = ['Actual Code', 
                                  'Predicted Codes',
                                  'Similarity Codes'])


threshold_list = [0.6, 0.7, 0.8, 0.9]

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)

    data_input_format = []

    i = 0
    for i in range(m):
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_codes = [str(y) for y in Y[i][2:-2].split("', '") if y in icd_codes]#To get rid of '' and ,
        act_codes_string = ', '.join(act_codes)

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh


        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred_indices = np.nonzero(sim_post_threshold)[0].tolist()

        #Get codes from indices
        pred_codes = [str(dict_index_code[p]) for p in pred_indices]
        #Convert to Comma Separated String
        pred_codes_string = ', '.join(pred_codes)
        #print([pred_codes_string])

        #Corresponding Similarity Scores of List of Indices
        sim_scores = [str(sim[i][j]) for j in pred_indices]
        #Convert to Comma Separated String
        sim_scores_string = ', '.join(sim_scores)
        #print([sim_scores_string])


        data_input = [act_codes_string, pred_codes_string, sim_scores_string]

        #Append
        data_input_format.append(data_input)

    df = pd.DataFrame(data_input_format, columns = ['Actual Codes', 'Predicted Codes', 'Similarity Scores'])
    df.to_csv('Similarity_Data/Code and Similarity Data - Unsupervised ICD - RAMN - ' + str(m) + ' Records_' + str(thresh) + '_threshold.csv')


Commenced for Similarity Threshold: 0.6

Commenced for Similarity Threshold: 0.7

Commenced for Similarity Threshold: 0.8

Commenced for Similarity Threshold: 0.9


In [38]:
#Initialize Results Dataframe which will store the relevant metrics for each threshold
results = pd.DataFrame(columns = ['Number of Observations', 
                                  'Number of ICD Codes',
                                  'Threshold',
                                  'Precision', 
                                  'Recall', 
                                  'F1-Score', 
                                  'Specificity'])

#Compute Accuracy metrics based on thresholding        
#threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 
#                  0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
threshold_list = [0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.4]
#threshold_list = [1.0, 5.0 10.0, 15.0, 20.0]


for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)
    
    #X = DD_Data['clean_text'][0]
    #Y = list(DD_Data['ICD9_CODE'])
    
    predicted_codes_index = []
    actual_codes_index = []

    #m = 1#len(Y)

    for i in range(m):
        #text = X#[i]
        #print(i)
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_icd_indices = [dict_code_index[y] for y in Y[i][2:-2].split("', '") if y in icd_codes] #To get rid of '' and ,

        #Initialize Prediction list. This will contain indices of similar icd codes
        pred = []


        #sent_emb = get_sentence_embedding(text, word_model)

        #Modified#Calculate cosine similarity of sentence with icd sentences
        #sim = np.array([RAMN_similarity(text, icd_text, word_model, freq_vec) for icd_text in icd_sentences])#cosine_sim1_matrix(sent_emb)

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh

        
        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred.extend(np.nonzero(sim_post_threshold)[0].tolist())

        #Consider the unique indices
        pred = set(pred)

        #Convert list of indices to hot vector
        pred_icd_vector = generate_binary(pred)
        act_icd_vector = generate_binary(act_icd_indices)

        #Append
        predicted_codes_index.append(pred_icd_vector)
        actual_codes_index.append(act_icd_vector)


        #if i%100 == 0 and i != 0:
        #    print(i, ' sentences done.')

    #Calculate Precision, Recall, F1 Score for this threshold
    precision, recall, f1_score, support = precision_recall_fscore_support(actual_codes_index, predicted_codes_index, average = 'micro')
    specificity = calc_specificity(predicted_codes_index, actual_codes_index)

    #Add to results Dataframe:
    results = results.append({'Number of Observations': m, 
                    'Number of ICD Codes': NUM_ICD_CODES, 
                    'Threshold': thresh,
                    'Precision': precision, 
                    'Recall': recall, 
                    'F1-Score': f1_score, 
                    'Specificity': specificity}, ignore_index=True)

    print('\nPrecision: ', precision, ' Recall: ', recall, ' F1-Score: ', f1_score, ' Specificity:', specificity)


time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')    
    
#Print Results
#RESULT_FILENAME = 'Results/Experiment_Results_RAMN_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
RESULT_FILENAME = 'Experiment_Results_RAMN_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
results.to_excel(RESULT_FILENAME) 
print('Results Saved!')
results


Commenced for Similarity Threshold: 0.6

Precision:  0.018145166227132598  Recall:  0.11910664989693041  F1-Score:  0.03149262461027332  Specificity: 0.9749778644137431

Commenced for Similarity Threshold: 0.7

Precision:  0.030802228490567806  Recall:  0.10566139330791084  F1-Score:  0.047699252539379774  Specificity: 0.9870907906456254

Commenced for Similarity Threshold: 0.8

Precision:  0.04903753550573578  Recall:  0.09698184930543556  F1-Score:  0.06513862368174243  Specificity: 0.9926962898283516

Commenced for Similarity Threshold: 0.9

Precision:  0.05665533785038726  Recall:  0.09438200441210154  F1-Score:  0.0708069179034062  Specificity: 0.9938968739718222

Time elapsed:  18468.255376652876  minutes
Results Saved!


,Number of Observations,Number of ICD Codes,Threshold,Precision,Recall,F1-Score,Specificity
0,22722.0,2833.0,0.6,0.018145,0.119107,0.031493,0.974978
1,22722.0,2833.0,0.7,0.030802,0.105661,0.047699,0.987091
2,22722.0,2833.0,0.8,0.049038,0.096982,0.065139,0.992696
3,22722.0,2833.0,0.9,0.056655,0.094382,0.070807,0.993897
